In [36]:
!pip3 install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub
# !pip3 install pyAudioAnalysis
# !pip3 install plot_metrics
!pip3 install keras
!pip3 install boto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import all the required libraries

In [37]:
import fnmatch
import os
import zipfile
import re

import scipy.io.wavfile as wavfile
import wave

import numpy as np
from numpy.lib import stride_tricks
import os
import tarfile
from PIL import Image
import scipy.io.wavfile as wav

import pandas as pd

import random
import boto

from sklearn.metrics import confusion_matrix, roc_curve, auc
# from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
%cd /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data

/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data


In [40]:
# !wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --no-check-certificate https://dcapswoz.ict.usc.edu/wwwedaic/data/

In [41]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data'

# directory where audio and transcripts folders will be created
out_dir = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw'

In [42]:
# !wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --no-check-certificate https://dcapswoz.ict.usc.edu/wwwedaic/labels/

In [43]:
# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv')

# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv')

# /content/drive/MyDrive/Depression_Analysis_Research/dev_split.csv
dev_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/dev_split.csv')

In [44]:
print(train_df.shape)
print(test_df.shape)
print(dev_df.shape)

(163, 6)
(56, 6)
(56, 6)


In [45]:
print(train_df.head())
print(test_df.head())
print(dev_df.head())

   Participant_ID   Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             302     male           0          4             0             28
1             303   female           0          0             0             17
2             304   female           0          6             0             20
3             305     male           0          7             0             28
4             307  female            0          4             0             23
   Participant_ID  Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             600  female           0          5             0           23.0
1             602  female           1         13             1           67.0
2             604    male           1         12             0           30.0
3             605    male           0          2             0           23.0
4             606  female           0          5             0           46.0
   Participant_ID  Gender  PHQ_Binary  PHQ_Score  PCL-C (P

In [46]:
def in_split(partic_id, df):
    """
    Returns True if the participant is in the AVEC training split
    (aka participant's we have depression labels for)
    """
    return partic_id in set(df['Participant_ID'].values)

def get_depression_label(partic_id, df):
    """
    Returns participant's PHQ8 Binary label. 1 representing depression;
    0 representing no depression.
    """
    return df.loc[df['Participant_ID'] == partic_id]['PHQ_Binary'].item()


In [47]:
def build_class_dictionaries(dir_name, df):
    
    print("dir name: " + dir_name)

    depressed_dict = dict()
    normal_dict = dict()
    
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if fnmatch.fnmatch(file, '*BoAW_openSMILE_2.3.0_eGeMAPS.csv'):
                # print(file)
                regex = re.compile(r'\d+')
                partic_id = [int(x) for x in regex.findall(file)][0]

                if in_split(partic_id, df):
                    file_name = dir_name + '/' + str(partic_id) + '_P/features/' + file
                    # %cd dir_name
                    # print(type(file))
                    df_1 = pd.read_csv(file_name, header=None)
                    df_1.drop(0, axis=1, inplace=True)
                    mat = np.mat(df_1)

                    depressed = get_depression_label(partic_id, df)  # 1 if True
                    # print(depressed)
                    if depressed:
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
            # break
    return depressed_dict, normal_dict

In [48]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features'
depressed_train_dict, normal_train_dict = build_class_dictionaries(dir_name, train_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features


In [49]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features'
depressed_test_dict, normal_test_dict = build_class_dictionaries(dir_name, test_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features


In [50]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features'
depressed_dev_dict, normal_dev_dict = build_class_dictionaries(dir_name, dev_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/aud_features


In [51]:
del(dir_name)

In [52]:
print(len(depressed_train_dict))
print(len(normal_train_dict))

print(len(depressed_test_dict))
print(len(normal_test_dict))

print(len(depressed_dev_dict))
print(len(normal_dev_dict))

37
126
17
39
12
44


In [53]:
def create_y_set(depressed_dict, normal_dict, df):
    """
    Building an array that describes if a participant is depressed or not
    """
    merged_dict = {**depressed_dict, **normal_dict}

    y_func = list()
    part_id = list()

    for id in df['Participant_ID']:
        part_id.append(id)

    # print(len(part_id))
    for key, value in merged_dict.items():
        if key in part_id:
            x = df.loc[df['Participant_ID']==key, 'PHQ_Binary'].iloc[0]
            y_func.append(x)    
    # print(df['Participant_ID'])
    # y_func = pd.DataFrame()
    return y_func, merged_dict



In [54]:
y_train, merged_train_dict = create_y_set(depressed_train_dict, normal_train_dict, train_df)
del(depressed_train_dict)
del(normal_train_dict)

In [55]:
y_test, merged_test_dict = create_y_set(depressed_test_dict, normal_test_dict, test_df)
del(depressed_test_dict)
del(normal_test_dict)

In [56]:
y_dev, merged_dev_dict = create_y_set(depressed_dev_dict, normal_dev_dict, dev_df)
del(depressed_dev_dict)
del(normal_dev_dict)

In [57]:
x_train = np.array(list(merged_train_dict.values()))
x_test = np.array(list(merged_test_dict.values()))
x_dev = np.array(list(merged_dev_dict.values()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify

In [58]:
del(merged_train_dict)
del(merged_test_dict)
del(merged_dev_dict)

In [59]:
print(x_train[0].shape)
print(x_test[0].shape)
print(x_dev[0].shape)

(8396, 101)
(7779, 0)
(8118, 101)


In [60]:
def data_handling(x, y):

    for i in range(len(x)):
        x[i] = np.ravel(x[i])

    max = -1
    for i in range(len(y)):
        if len(y[i]) > max:
            max = len(y[i])
    
    for i in range(len(x)):
        length = max - len(x[i])
        x[i] = np.pad(x[i], (0, length), 'constant')
        del(length)

    arr = np.array([])
    for element in x:
        arr = np.concatenate((arr, element))
        del(element)
    
    del(x)

    return arr, max

In [61]:
x_train_1 = x_train[0:81] 
x_train_2 = x_train[81:163]

arr_1, max_train = data_handling(x_train_1, x_train)
del(x_train_1)

arr_2, max_train = data_handling(x_train_2, x_train)
del(x_train_2)

del(x_train)

arr_1 = np.concatenate((arr_1, arr_2))

del(arr_2)

arr_3, max_test = data_handling(x_test, x_test)
del(x_test)

arr_4, max_dev = data_handling(x_dev, x_dev)
del(x_dev)

In [62]:
x_train = arr_1.reshape(163, int(max_train/101), 101)
del(arr_1)

x_test = arr_3.reshape(56, int(max_test/101), 101)
del(arr_3)

x_dev = arr_4.reshape(56, int(max_dev/101), 101)
del(arr_4)

In [63]:
del(max_train)
del(max_test)
del(max_dev)

In [64]:
print(x_train.shape)
print(x_test.shape)
print(x_dev.shape)

(163, 19579, 101)
(56, 0, 101)
(56, 29679, 101)


In [65]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_dev = np.array(y_dev)

In [66]:
# create audio directory
out_dir = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data'
vid_features_dir = os.path.join(out_dir, 'aud_features_BoAW_openSMILE_230_eGeMAPS')
if not os.path.exists(vid_features_dir):
    os.makedirs(vid_features_dir)

In [67]:
%ls
%cd aud_features_BoAW_openSMILE_2.3.0_eGeMAPS
%ls

aud_features_BoAW_openSMILE_2.3.0_eGeMAPS/
aud_features_BoAW_openSMILE_230_eGeMAPS/
aud_features_BoAW_openSMILE_230_MFCC/
raw/
vid_features_BoVW_openFace_210_Pose_Gaze_AUs/
vid_features_OpenFace_210_Pose_Gaze_AUs/
/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/aud_features_BoAW_openSMILE_2.3.0_eGeMAPS


In [68]:
filename = 'x_train_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('x_train_BoAW_openSMILE_230_eGeMAPS.gz', x_train.reshape(x_train.shape[0], -1), delimiter=',')
del(x_train)

filename = 'x_test_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('x_test_BoAW_openSMILE_230_eGeMAPS.gz', x_test.reshape(x_test.shape[0], -1), delimiter=',')
del(x_test)

filename = 'x_dev_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('x_dev_BoAW_openSMILE_230_eGeMAPS.gz', x_dev.reshape(x_dev.shape[0], -1), delimiter=',')
del(x_dev)

In [69]:
filename = 'y_train_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('y_train_BoAW_openSMILE_230_eGeMAPS.gz', y_train.reshape(y_train.shape[0], -1), delimiter=',')

filename = 'y_test_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('y_test_BoAW_openSMILE_230_eGeMAPS.gz', y_test.reshape(y_test.shape[0], -1), delimiter=',')

filename = 'y_dev_BoAW_openSMILE_230_eGeMAPS.gz'
np.savetxt('y_dev_BoAW_openSMILE_230_eGeMAPS.gz', y_dev.reshape(y_dev.shape[0], -1), delimiter=',')